In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  


In [2]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [3]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
            "mean_40", "variance_40",
            "mean_50", "variance_50", "cap"]

rows = []

In [5]:
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor() ]))  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()]))

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= class_count[0]

[5000   50]


In [7]:
beta = 0.999 

exp = np.empty_like(targets)
for i, count in enumerate(class_count):
    exp[targets==i] = count
effective_weights = (1 - beta) / ( 1 - (beta ** torch.from_numpy(exp)))

In [8]:
train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [9]:
# 2 CLASS normal

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.001968830943107605, AUC: 0.6361665000000001


Test set: Avg. loss: 0.0006069920063018799, AUC: 0.8331379999999999


Test set: Avg. loss: 0.0005906819999217987, AUC: 0.849458



KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])


df2.to_csv('results/convnet_aucs.csv', index=False)

rows = []


In [ ]:
# 2 CLASS ratio

momentum=0
learning_rates = [1e-3]

learning_rate_aucs = []
learning_rate_train_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.001968830943107605, AUC: 0.6361665000000001


Test set: Avg. loss: 0.0019515877962112426, AUC: 0.7094229999999999


Test set: Avg. loss: 0.0008754172098666134, AUC: 0.743984


Test set: Avg. loss: 0.0016984249353408814, AUC: 0.7375505000000001


Test set: Avg. loss: 0.0009006779843775352, AUC: 0.7743800000000001


Test set: Avg. loss: 0.0015235986113548278, AUC: 0.7784579999999999


Test set: Avg. loss: 0.0009531024183229644, AUC: 0.8180480000000001


Test set: Avg. loss: 0.0031965354681015015, AUC: 0.4105075


Test set: Avg. loss: 0.0026761832237243654, AUC: 0.610581


Test set: Avg. loss: 0.0009113745841354427, AUC: 0.636804


Test set: Avg. loss: 0.002043037295341492, AUC: 0.6594949999999999


Test set: Avg. loss: 0.0008553141438503667, AUC: 0.71688


Test set: Avg. loss: 0.001632628858089447, AUC: 0.6885979999999999


Test set: Avg. loss: 0.0009908383394969572, AUC: 0.7618640000000001


Test set: Avg. loss: 0.0026044973134994507, AUC: 0.512846


Test set: Av

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [31]:
# 2 CLASS oversampled

momentum=0
learning_rates = [1e-3]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)




Test set: Avg. loss: 0.0009614106118679046, AUC: 0.614011


Test set: Avg. loss: 0.0006774698793888092, AUC: 0.6585589999999999


Test set: Avg. loss: 0.0006662350296974182, AUC: 0.713206


Test set: Avg. loss: 0.0006555393636226654, AUC: 0.7449055


Test set: Avg. loss: 0.006931622505187988, AUC: 0.532071


Test set: Avg. loss: 0.0006511834263801575, AUC: 0.705865


Test set: Avg. loss: 0.0006510878801345825, AUC: 0.7048525


Test set: Avg. loss: 0.0006323479413986206, AUC: 0.74831


Test set: Avg. loss: 0.0012074116468429566, AUC: 0.45304300000000003


Test set: Avg. loss: 0.0006792399287223815, AUC: 0.6182110000000001


Test set: Avg. loss: 0.0006664896011352539, AUC: 0.667305


Test set: Avg. loss: 0.0006620110869407654, AUC: 0.6808749999999999


Test set: Avg. loss: 0.005842076301574707, AUC: 0.35606649999999995


Test set: Avg. loss: 0.0006702626347541809, AUC: 0.6863610000000001


Test set: Avg. loss: 0.0006543796062469482, AUC: 0.7483445000000001


Test set: Avg. loss: 0.00063

In [32]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [33]:
# 2 CLASS undersampled

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.002664048075675964, AUC: 0.35103100000000004


Test set: Avg. loss: 0.0008118733763694763, AUC: 0.39093300000000003


Test set: Avg. loss: 0.0007359330058097839, AUC: 0.42988400000000004


Test set: Avg. loss: 0.0007080410122871399, AUC: 0.476849


Test set: Avg. loss: 0.005757878303527832, AUC: 0.4101775


Test set: Avg. loss: 0.0008514148592948913, AUC: 0.663575


Test set: Avg. loss: 0.0006793743073940277, AUC: 0.6388240000000001


Test set: Avg. loss: 0.0006851064562797546, AUC: 0.599984


Test set: Avg. loss: 0.001174522042274475, AUC: 0.498127


Test set: Avg. loss: 0.0009026037454605102, AUC: 0.569188


Test set: Avg. loss: 0.0007478025853633881, AUC: 0.5202549999999999


Test set: Avg. loss: 0.0007039127349853516, AUC: 0.5297069999999999


Test set: Avg. loss: 0.0009038377702236175, AUC: 0.47197


Test set: Avg. loss: 0.0007971824407577515, AUC: 0.48862799999999995


Test set: Avg. loss: 0.0007626304626464844, AUC: 0.494027


Test set: Avg. loss: 0.00074

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [ ]:
# 2 Class Weighted Loss 

momentum=0
learning_rates = [1e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['pos_weight'] = torch.tensor([weight[1]])


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


In [34]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [9]:
# 2 CLASS SMOTE

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.001968830943107605, AUC: 0.6361665000000001



KeyboardInterrupt: 

In [36]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [16]:
# 2 Class Capped SMOTE 

momentum=0
learning_rates = [1e-3, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []
    

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_sigmoid(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args, smote=True)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for i in range(len(cap_aucs)):
    auc_mean = cap_aucs[i][0]
    auc_variance = cap_aucs[i][1]
    cap = caps[i]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0015577556490898132, AUC: 0.6859669999999999


Test set: Avg. loss: 0.0009647067785263061, AUC: 0.8755560000000001


Test set: Avg. loss: 0.0006530835032463074, AUC: 0.9021619999999999


Test set: Avg. loss: 0.0007020677328109741, AUC: 0.8893929999999999


Test set: Avg. loss: 0.001078621029853821, AUC: 0.423891


Test set: Avg. loss: 0.0004322097599506378, AUC: 0.889239


Test set: Avg. loss: 0.00042063261568546295, AUC: 0.89573


Test set: Avg. loss: 0.000940508782863617, AUC: 0.898558


Test set: Avg. loss: 0.003058913707733154, AUC: 0.49637050000000005


Test set: Avg. loss: 0.0008342242240905761, AUC: 0.853572


Test set: Avg. loss: 0.0010090836882591249, AUC: 0.8785789999999999


Test set: Avg. loss: 0.001222649872303009, AUC: 0.8876470000000001


Test set: Avg. loss: 0.003471907377243042, AUC: 0.5334


Test set: Avg. loss: 0.0016670249700546265, AUC: 0.735833


Test set: Avg. loss: 0.0005569218099117279, AUC: 0.8988700000000001


Test set: Avg. loss: 0.00


Test set: Avg. loss: 0.00057944455742836, AUC: 0.8650279999999999


Test set: Avg. loss: 0.0019157962203025817, AUC: 0.5303475


Test set: Avg. loss: 0.0006247777640819549, AUC: 0.8047059999999999


Test set: Avg. loss: 0.000615023136138916, AUC: 0.8285035000000001


Test set: Avg. loss: 0.0006052527129650116, AUC: 0.8387680000000003


Test set: Avg. loss: 0.0031466708183288572, AUC: 0.416586


Test set: Avg. loss: 0.0005910550951957703, AUC: 0.844524


Test set: Avg. loss: 0.0005934758484363556, AUC: 0.849567


Test set: Avg. loss: 0.0005860161185264587, AUC: 0.85896


Test set: Avg. loss: 0.006763181924819946, AUC: 0.6645800000000001


Test set: Avg. loss: 0.0006936165690422058, AUC: 0.4969805


Test set: Avg. loss: 0.0006934365332126617, AUC: 0.49798800000000004


Test set: Avg. loss: 0.0006933400630950928, AUC: 0.4970085


Test set: Avg. loss: 0.011637805938720703, AUC: 0.4944835


Test set: Avg. loss: 0.0006524167358875275, AUC: 0.770158


Test set: Avg. loss: 0.00064747983217239

NameError: name 'cap_auc' is not defined

In [21]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = (col_names[0:13].append(col_names[17])))

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [37]:
# 2 CLASS Focal Loss

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['reduction'] = 'mean'




for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SigmoidFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.001035837471485138, AUC: 0.44774200000000003


Test set: Avg. loss: 0.0006927213966846466, AUC: 0.5254145


Test set: Avg. loss: 0.0006931669712066651, AUC: 0.511691


Test set: Avg. loss: 0.0006935113668441773, AUC: 0.5110125


Test set: Avg. loss: 0.005613439321517945, AUC: 0.426767


Test set: Avg. loss: 0.0037078361511230467, AUC: 0.5171749999999999


Test set: Avg. loss: 0.0026477036476135253, AUC: 0.46210599999999996


Test set: Avg. loss: 0.002063918948173523, AUC: 0.44280450000000005


Test set: Avg. loss: 0.001599363923072815, AUC: 0.5595060000000001


Test set: Avg. loss: 0.0017810710668563843, AUC: 0.527251


Test set: Avg. loss: 0.0017536671757698058, AUC: 0.530268


Test set: Avg. loss: 0.001468895673751831, AUC: 0.511718


Test set: Avg. loss: 0.0009869316220283508, AUC: 0.521323


Test set: Avg. loss: 0.0025733143091201784, AUC: 0.44177999999999995


Test set: Avg. loss: 0.002441696286201477, AUC: 0.42422899999999997


Test set: Avg. loss: 0.00228

In [38]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

NameError: name 'df2' is not defined

In [ ]:
# distance + capped loss
momentum=0
learning_rates = [1e-3, 1e-4]


    
loss_fn_args = {}
loss_fn_args['loss_cap'] = None

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNetWithEmbeddings(2)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args, smote=True)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)


for i in range(len(learning_rates)): 
    row = ["capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)

In [39]:
df2.head(50)

,name,num_classes,classes_used,ratio,learning_rate,mean_0,variance_0,mean_10,variance_10,mean_20,variance_20,mean_30,variance_30
0,focal_loss,2,"(0, 1)","(100, 1)",0.00010,0.487865,0.013197,0.489767,0.003418,0.476831,0.003291,0.469356,0.003572
1,focal_loss,2,"(0, 1)","(100, 1)",0.00001,0.562734,0.009148,0.578999,0.008085,0.571636,0.009897,0.567872,0.010230


In [33]:
NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (20, 2, 1)


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED)


weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [8]:
# 3 class normal

learning_rates = [1e-4, 1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


Test set: Avg. loss: 0.004159178098042806, AUC: 0.5



KeyboardInterrupt: 

In [11]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [15]:
#  3 class ratio

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


Test set: Avg. loss: 0.0028148902257283527, AUC: 0.5499715000000001


Test set: Avg. loss: 0.0012899534304936728, AUC: 0.5


Test set: Avg. loss: 0.0011812520821889241, AUC: 0.5


Test set: Avg. loss: 0.0011003198623657227, AUC: 0.5


Test set: Avg. loss: 0.0012160149812698365, AUC: 0.496695


Test set: Avg. loss: 0.0012602541049321493, AUC: 0.5


Test set: Avg. loss: 0.0011723772684733072, AUC: 0.5


Test set: Avg. loss: 0.0012640552123387655, AUC: 0.5


Test set: Avg. loss: 0.001084180474281311, AUC: 0.47396475


Test set: Avg. loss: 0.0012727203766504925, AUC: 0.4995


Test set: Avg. loss: 0.0010816088120142618, AUC: 0.5045000000000001


Test set: Avg. loss: 0.0011214701334635417, AUC: 0.5035


Test set: Avg. loss: 0.0026806603272755943, AUC: 0.49925


Test set: Avg. loss: 0.0012301311095555623, AUC: 0.5


Test set: Avg. loss: 0.001151394208272298, AUC: 0.5


Test set: Avg. loss: 0.00107947838306427, AUC: 0.5


Test set: Avg. loss: 0.0020958302021026613, AUC: 0.44825000000000004




In [16]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [17]:
# 3 class oversampled 

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.003038533369700114, AUC: 0.5


Test set: Avg. loss: 0.0010433460474014282, AUC: 0.574998


Test set: Avg. loss: 0.0011814462741216024, AUC: 0.7483075


Test set: Avg. loss: 0.001038165827592214, AUC: 0.7227574999999999


Test set: Avg. loss: 0.007914267698923746, AUC: 0.49925


Test set: Avg. loss: 0.0010645556449890136, AUC: 0.37466975


Test set: Avg. loss: 0.001031708836555481, AUC: 0.6914997500000001


Test set: Avg. loss: 0.0011311002572377523, AUC: 0.75428325


Test set: Avg. loss: 0.006719241301218669, AUC: 0.5075000000000001


Test set: Avg. loss: 0.0010695095856984456, AUC: 0.5922592499999999


Test set: Avg. loss: 0.0010078495343526204, AUC: 0.66097125


Test set: Avg. loss: 0.0009656443595886231, AUC: 0.68574


Test set: Avg. loss: 0.002060540755589803, AUC: 0.5615957500000001


Test set: Avg. loss: 0.0010401540199915568, AUC: 0.3627755


Test set: Avg. loss: 0.0009678711692492167, AUC: 0.6725415


Test set: Avg. loss: 0.0009192776083946228, AUC: 0.67

In [18]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [19]:
# 3 class undersampled  

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0020289310614267984, AUC: 0.4756965


Test set: Avg. loss: 0.001091553012530009, AUC: 0.48456375


Test set: Avg. loss: 0.0010886570612589517, AUC: 0.4860155


Test set: Avg. loss: 0.0010853137572606406, AUC: 0.5097382500000001


Test set: Avg. loss: 0.0019787512222925823, AUC: 0.57222475


Test set: Avg. loss: 0.0011532610654830932, AUC: 0.49576575000000006


Test set: Avg. loss: 0.001145702044169108, AUC: 0.4937625


Test set: Avg. loss: 0.001140582799911499, AUC: 0.49350675


Test set: Avg. loss: 0.005084774017333984, AUC: 0.5


Test set: Avg. loss: 0.0010802105665206909, AUC: 0.50175


Test set: Avg. loss: 0.0010755322376887005, AUC: 0.502


Test set: Avg. loss: 0.0010662730137507121, AUC: 0.5022505


Test set: Avg. loss: 0.009249690055847169, AUC: 0.49975


Test set: Avg. loss: 0.0010924884875615438, AUC: 0.50599775


Test set: Avg. loss: 0.0010882760683695474, AUC: 0.5044992500000001


Test set: Avg. loss: 0.0010864359935124715, AUC: 0.50449775


Test set:

In [20]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = [] 

In [21]:
#  3 class weighted

learning_rates = [1e-3]

learning_rate_aucs = []

loss_fn_args={}
loss_fn_args['weight'] = torch.from_numpy(weight).float()


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.00608756939570109, AUC: 0.501


Test set: Avg. loss: 0.0010771948893864949, AUC: 0.5007499999999999


Test set: Avg. loss: 0.001078349749247233, AUC: 0.49949999999999994


Test set: Avg. loss: 0.0010784133275349936, AUC: 0.50025


Test set: Avg. loss: 0.00372534704208374, AUC: 0.50023525


Test set: Avg. loss: 0.0010370986064275106, AUC: 0.49925


Test set: Avg. loss: 0.0010473136504491171, AUC: 0.49649774999999996


Test set: Avg. loss: 0.0010565840800603231, AUC: 0.5224875


Test set: Avg. loss: 0.0020539817810058592, AUC: 0.50625


Test set: Avg. loss: 0.0010878965854644776, AUC: 0.50025


Test set: Avg. loss: 0.001087069312731425, AUC: 0.49999999999999994


Test set: Avg. loss: 0.0010845698912938435, AUC: 0.50025


Test set: Avg. loss: 0.006659661134084066, AUC: 0.49401975000000004


Test set: Avg. loss: 0.0010928993225097657, AUC: 0.53216375


Test set: Avg. loss: 0.0010782272815704345, AUC: 0.54923675


Test set: Avg. loss: 0.0010686718225479126, AUC: 0.55

In [22]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = [] 

In [23]:
#  3 class focal loss

learning_rates = [1e-3]

learning_rate_aucs = []

loss_fn_args={}
loss_fn_args['reduction'] = 'mean'


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SoftmaxFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0011305590867996215, AUC: 0.4987425


Test set: Avg. loss: 0.001101138710975647, AUC: 0.5


Test set: Avg. loss: 0.0011463310718536376, AUC: 0.5


Test set: Avg. loss: 0.0011780770619710286, AUC: 0.5


Test set: Avg. loss: 0.0038984591166178386, AUC: 0.5


Test set: Avg. loss: 0.0011603130102157593, AUC: 0.5


Test set: Avg. loss: 0.0011915287176767985, AUC: 0.5


Test set: Avg. loss: 0.0012100194692611695, AUC: 0.5


Test set: Avg. loss: 0.006762343406677246, AUC: 0.50025


Test set: Avg. loss: 0.001145007332166036, AUC: 0.5


Test set: Avg. loss: 0.0011761978069941203, AUC: 0.5


Test set: Avg. loss: 0.0011957573493321736, AUC: 0.5


Test set: Avg. loss: 0.0017103184858957927, AUC: 0.4808077500000001


Test set: Avg. loss: 0.0011710822582244873, AUC: 0.50025


Test set: Avg. loss: 0.0012000585397084554, AUC: 0.50025


Test set: Avg. loss: 0.0012189826170603433, AUC: 0.50025


Test set: Avg. loss: 0.003416938861211141, AUC: 0.5


Test set: Avg. loss: 0.00113592

In [24]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [34]:
#  3 class SMOTE

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.004159178098042806, AUC: 0.5


Test set: Avg. loss: 0.0010713002681732179, AUC: 0.49825


Test set: Avg. loss: 0.0010870064496994018, AUC: 0.49875


Test set: Avg. loss: 0.0010937161048253376, AUC: 0.4980015


Test set: Avg. loss: 0.001217594623565674, AUC: 0.5495857500000001


Test set: Avg. loss: 0.0010883294343948364, AUC: 0.54002475


Test set: Avg. loss: 0.0009588491717974345, AUC: 0.6908785


Test set: Avg. loss: 0.0010892333984375, AUC: 0.6341857500000001


Test set: Avg. loss: 0.0012967588504155477, AUC: 0.44993225000000003


Test set: Avg. loss: 0.0010879942576090494, AUC: 0.49974900000000005


Test set: Avg. loss: 0.0010939101775487264, AUC: 0.4984995


Test set: Avg. loss: 0.0010963983138402302, AUC: 0.4980035


Test set: Avg. loss: 0.009516664822896322, AUC: 0.4979995


Test set: Avg. loss: 0.0010874385436375935, AUC: 0.502996


Test set: Avg. loss: 0.0010935383637746174, AUC: 0.50224575


Test set: Avg. loss: 0.0010963813463846842, AUC: 0.50149875



In [35]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []